In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import time
from datetime import datetime

# --- PARÁMETROS ESPECÍFICOS ---
GENRES = ['reggaeton']
YEARS = [2010, 2024] 

In [14]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')


In [15]:
# Años para iterar
YEARS = [2010, 2024] 

# Términos optimizados para una búsqueda de ALTO VOLUMEN
# Incluye etiquetas históricas ('dembow') y modernas ('trap latino', 'urbano latino')
query_terms = [
    'reggaeton',
    'regueton',
    'urbano latino',
    'perreo',
    'reggaeton pop',
    'latin urban',
    'pop latino',
    'trap latino',
    'urbano latino',
    'dembow',
    '"hip hop latino"' # Entre comillas para buscar la frase exacta (útil para el 2010)
]

# 4. Inicializa el cliente de Spotify
try:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")
    sp = None

✅ Conexión a Spotify exitosa.


In [26]:
#Comprobación de autenticación
print(type(sp)) 

<class 'spotipy.client.Spotify'>


In [16]:
# --- CELDA 2: DEFINICIÓN DE LA FUNCIÓN ESTABLE CON DOBLE LÍMITE (500 TOTAL, 250 ANUAL) ---

import time 
import pandas as pd 

def extract_spotify_data_simplificada(sp, years, query_terms): 
    """
    VERSION ESTABLE CON LÍMITES: 500 total, 250 por año. Extrae datos básicos
    y la Popularidad (sin Audio Features) para garantizar estabilidad.
    """
    
    if sp is None:
        print("Error: Cliente de Spotify no inicializado.")
        return pd.DataFrame()

    spotify_data = []
    MAX_TOTAL_SONGS = 500     # Límite global
    MAX_SONGS_PER_YEAR = 250  # Límite por año
    
    for year in years:
        
        # 1. VERIFICACIÓN DEL LÍMITE GLOBAL
        if len(spotify_data) >= MAX_TOTAL_SONGS:
            print(f"🛑 LÍMITE GLOBAL ALCANZADO ({MAX_TOTAL_SONGS}). Terminando la extracción.")
            break
            
        year_range = f'{year}-{year}'
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")
        
        songs_added_in_year = 0 # Contador reiniciado para el límite anual
        
        for term in query_terms:
            
            # 2. VERIFICACIÓN DEL LÍMITE ANUAL (antes de la búsqueda)
            if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                print(f"🛑 LÍMITE ANUAL ALCANZADO ({MAX_SONGS_PER_YEAR}) para {year}. Pasando al siguiente año.")
                break # Rompe el bucle de términos, y avanza al siguiente año.
            
            query = f'{term} year:{year_range}'
            print(f"   -> Buscando con término: {term}")
            
            # --- BLOQUE DE BÚSQUEDA ---
            try:
                results = sp.search(q=query, type='track', limit=50)
            except Exception as e:
                print(f"   ❌ ERROR grave al buscar con '{term}': {e}. Esperando 5s.")
                time.sleep(5)
                continue
            
            songs_added_in_term = 0
            
            while results: 
                
                for track in results['tracks']['items']:
                    
                    track_id = track['id']
                    
                    # 3. VERIFICACIÓN DE LÍMITES Y DEDUPLICACIÓN
                    if len(spotify_data) >= MAX_TOTAL_SONGS:
                        print(f"🛑 LÍMITE GLOBAL ALCANZADO ({MAX_TOTAL_SONGS}). Terminando la extracción.")
                        return pd.DataFrame(spotify_data)
                        
                    # 4. VERIFICACIÓN DE LÍMITE ANUAL (por si se alcanza con la canción actual)
                    if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                        break # Rompe el bucle de canciones (for track...), pasará al chequeo de abajo.
                        
                    if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                        continue
                        
                    # 5. Añadir la canción
                    data_row = {
                        'Artista': track['artists'][0]['name'],
                        'Género musical': f'Reggaeton (Origen: {term})',
                        'Tipo': 'Canción',
                        'Nombre': track['name'],
                        'Año de lanzamiento': year,
                        'ID_Spotify': track_id,
                        'ID_Album': track['album']['id'],
                        'Popularidad': track.get('popularity', None) 
                    }
                    spotify_data.append(data_row)
                    songs_added_in_year += 1
                    songs_added_in_term += 1

                # Rompe el bucle de paginación si se alcanzó el límite anual en el bucle interior.
                if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                    break 

                # 6. Paginación
                try:
                    if results['tracks']['next']:
                        results = sp.next(results['tracks'])
                        time.sleep(1.0) # Pausa estable entre páginas de 50
                    else:
                        results = None
                except Exception as e:
                    print(f"   ❌ ERROR al obtener la página siguiente: {e}")
                    results = None

            print(f"      -> Añadidas {songs_added_in_term} canciones ÚNICAS de '{term}'.")
            
    return pd.DataFrame(spotify_data)

In [18]:
# --- CELDA 3: LLAMADA, EJECUCIÓN Y GUARDADO DEL CSV ---

if sp:
    # Llamamos a la versión simplificada y estable
    df_spotify_raw = extract_spotify_data_simplificada(sp, YEARS, query_terms) 
    
    print("\n--- RESULTADO FINAL DE EXTRACCIÓN ESTABLE ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)}") 
    print(df_spotify_raw[['Nombre', 'Artista', 'Popularidad']].head())
else:
    print("❌ El cliente 'sp' no está inicializado. No se pudo ejecutar la extracción.")


--- INICIANDO EXTRACCIÓN PARA EL AÑO 2010 ---
   -> Buscando con término: reggaeton
      -> Añadidas 100 canciones ÚNICAS de 'reggaeton'.
   -> Buscando con término: regueton
      -> Añadidas 18 canciones ÚNICAS de 'regueton'.
   -> Buscando con término: urbano latino
      -> Añadidas 34 canciones ÚNICAS de 'urbano latino'.
   -> Buscando con término: perreo
      -> Añadidas 69 canciones ÚNICAS de 'perreo'.
   -> Buscando con término: reggaeton pop
      -> Añadidas 23 canciones ÚNICAS de 'reggaeton pop'.
   -> Buscando con término: latin urban
      -> Añadidas 6 canciones ÚNICAS de 'latin urban'.
🛑 LÍMITE ANUAL ALCANZADO (250) para 2010. Pasando al siguiente año.

--- INICIANDO EXTRACCIÓN PARA EL AÑO 2024 ---
   -> Buscando con término: reggaeton
      -> Añadidas 100 canciones ÚNICAS de 'reggaeton'.
   -> Buscando con término: regueton
      -> Añadidas 86 canciones ÚNICAS de 'regueton'.
   -> Buscando con término: urbano latino
      -> Añadidas 60 canciones ÚNICAS de 'urbano 

In [19]:
# VERIFICACIÓN DE CANCIONES
print(f"Número de canciones encontradas en df_spotify_raw: {len(df_spotify_raw)}")
print(f"Primeras filas (head) de df_spotify_raw:\n{df_spotify_raw.head()}")

Número de canciones encontradas en df_spotify_raw: 500
Primeras filas (head) de df_spotify_raw:
          Artista                 Género musical     Tipo  \
0  Wisin & Yandel  Reggaeton (Origen: reggaeton)  Canción   
1  Wisin & Yandel  Reggaeton (Origen: reggaeton)  Canción   
2         Farruko  Reggaeton (Origen: reggaeton)  Canción   
3          Plan B  Reggaeton (Origen: reggaeton)  Canción   
4     Cosculluela  Reggaeton (Origen: reggaeton)  Canción   

                          Nombre  Año de lanzamiento              ID_Spotify  \
0                        Tu Olor                2010  4lwJGmQFcbUypQ8mkm8YlG   
1      Zun Zun Rompiendo Caderas                2010  5aJshF6mtr7SsztEMhAvZv   
2               Su Hija Me Gusta                2010  5N1EdvOZKDKKIcwBcj3VmR   
3  Parisera (feat. De La Ghetto)                2010  4rVeA8n8ui7neX4XUOJObT   
4                Te Deseo el Mal                2010  43hrb3UDtUdvVNQ5gSCsMI   

                 ID_Album  Popularidad  
0  4mIV2lIQfHsz

In [20]:
# --- CELDA COMPLETA: EXTRACCIÓN Y UNIFICACIÓN DE ÁLBUMES ---

import pandas as pd
import time

# -------------------------------------------------------------------
# BLOQUE 1: DEFINICIÓN DE LA FUNCIÓN
# -------------------------------------------------------------------

def extract_album_details(sp, album_ids):
    """
    Extrae los metadatos completos (nombre, año, artista) de los álbumes.
    """
    album_details = []
    
    # El set ya se crea cuando se llama a la función con album_ids_list.
    unique_album_ids = album_ids
    
    print(f"\n--- INICIANDO EXTRACCIÓN DE METADATOS DE {len(unique_album_ids)} ÁLBUMES ---")
    
    for album_id in unique_album_ids:
        try:
            album_info = sp.album(album_id) 
            
            data_row = {
                'ID_Album': album_id,
                'Nombre_Album': album_info['name'],
                'Año_Lanzamiento_Album': album_info['release_date'][:4], # Tomamos solo el año
                'Artista_Principal_Album': album_info['artists'][0]['name'],
                'Tipo_Lanzamiento': album_info['album_type'] 
            }
            album_details.append(data_row)
            time.sleep(0.2) # Pausa para no saturar la API
            
        except Exception as e:
            # Capturamos errores como álbumes que han sido eliminados de Spotify
            print(f"   -> Error al extraer detalles del álbum {album_id}: {e}")
            pass
            
    print(f"✅ Extracción de {len(album_details)} álbumes completada.")
    return pd.DataFrame(album_details)

# -------------------------------------------------------------------
# BLOQUE 2: EJECUCIÓN Y FUSIÓN
# -------------------------------------------------------------------

# Verificación de que el DataFrame de canciones está disponible
if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    print("\n--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---")
    
    # 1. VERIFICACIÓN CRÍTICA: La columna 'ID_Album' debe existir.
    if 'ID_Album' not in df_spotify_raw.columns:
        print("❌ ERROR CRÍTICO: La columna 'ID_Album' NO se encontró. La Celda 3 falló al guardar el ID del álbum.")
    else:
        
        # 2. Obtener IDs únicos válidos de los álbumes
        album_ids_list = df_spotify_raw['ID_Album'].dropna().unique() 
        
        if len(album_ids_list) == 0:
            print("❌ LISTA VACÍA: Se encontraron canciones, pero 0 IDs de álbumes válidos. No se puede continuar.")
        else:
            print(f"✅ Se encontraron {len(album_ids_list)} IDs de álbumes únicos. Extrayendo ahora...")
            
            # 3. Ejecutar la función de extracción (Tardará unos minutos)
            df_album_details = extract_album_details(sp, album_ids_list)
            
            # 4. Fusionar (JOIN) los detalles del álbum con los datos de las canciones
            df_spotify_final = pd.merge(
                df_spotify_raw,
                df_album_details[['ID_Album', 'Nombre_Album', 'Año_Lanzamiento_Album', 'Artista_Principal_Album']],
                on='ID_Album',
                how='left' 
            )
            
            print(f"\n--- PROCESO DE ÁLBUMES COMPLETADO ---")
            print(f"DataFrame intermedio 'df_spotify_final' creado con {len(df_spotify_final)} filas.")
            print(df_spotify_final.head())
            
else:
    print("❌ ERROR: El DataFrame de canciones (df_spotify_raw) no está disponible. Vuelve a ejecutar la Celda 3.")


--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---
✅ Se encontraron 290 IDs de álbumes únicos. Extrayendo ahora...

--- INICIANDO EXTRACCIÓN DE METADATOS DE 290 ÁLBUMES ---
✅ Extracción de 290 álbumes completada.

--- PROCESO DE ÁLBUMES COMPLETADO ---
DataFrame intermedio 'df_spotify_final' creado con 500 filas.
          Artista                 Género musical     Tipo  \
0  Wisin & Yandel  Reggaeton (Origen: reggaeton)  Canción   
1  Wisin & Yandel  Reggaeton (Origen: reggaeton)  Canción   
2         Farruko  Reggaeton (Origen: reggaeton)  Canción   
3          Plan B  Reggaeton (Origen: reggaeton)  Canción   
4     Cosculluela  Reggaeton (Origen: reggaeton)  Canción   

                          Nombre  Año de lanzamiento              ID_Spotify  \
0                        Tu Olor                2010  4lwJGmQFcbUypQ8mkm8YlG   
1      Zun Zun Rompiendo Caderas                2010  5aJshF6mtr7SsztEMhAvZv   
2               Su Hija Me Gusta                2010  5N1EdvOZKDKKIcwBcj3VmR 

In [21]:
def extract_lastfm_artist_details(artist_name):
    """Extrae biografía y estadísticas de popularidad de un artista."""
    
    url = 'http://ws.audioscrobbler.com/2.0/'
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': LASTFM_API_KEY,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'artist' in data:
            artist_info = data['artist']
            
            # Extraer los datos relevantes (Biografía, Popularidad/Plays)
            biography = artist_info.get('bio', {}).get('summary', 'N/A')
            playcount = artist_info.get('stats', {}).get('playcount', '0')
            listeners = artist_info.get('stats', {}).get('listeners', '0')
            
            return {
                'Artista': artist_name,
                'Biografia_Resumen': biography,
                'Playcount_LastFM': playcount,
                'Listeners_LastFM': listeners,
            }
    except Exception:
        # En caso de error de conexión o artista no encontrado, devuelve None
        pass 
        
    return None

# --- Ejecución para Artistas Únicos ---

if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    # Obtener la lista única de artistas para evitar búsquedas repetidas
    unique_artists = df_spotify_raw['Artista'].unique()
    lastfm_results = []
    
    print("\n--- EXTRACCIÓN DETALLES LAST.FM ---")
    
    for artist in unique_artists: 
        details = extract_lastfm_artist_details(artist)
        if details:
            lastfm_results.append(details)
        time.sleep(0.5) # Pausa crucial para no sobrecargar la API

    df_lastfm = pd.DataFrame(lastfm_results)
    
    # 5. UNIFICACIÓN DE DATOS
    # Fusionar los datos de Spotify con los detalles de Last.fm
    df_final_reggaeton = pd.merge(
        df_spotify_raw, 
        df_lastfm[['Artista', 'Playcount_LastFM', 'Listeners_LastFM']],
        on='Artista', 
        how='left' # Usamos left join para mantener todas las canciones de Spotify
    )
    
    print("\n--- DATAFRAME FINAL DE REGGAETON LISTO PARA ALMACENAMIENTO ---")
    print(df_final_reggaeton.head())
    print(f"Total de registros finales para Reggaeton: {len(df_final_reggaeton)}")


--- EXTRACCIÓN DETALLES LAST.FM ---

--- DATAFRAME FINAL DE REGGAETON LISTO PARA ALMACENAMIENTO ---
          Artista                 Género musical     Tipo  \
0  Wisin & Yandel  Reggaeton (Origen: reggaeton)  Canción   
1  Wisin & Yandel  Reggaeton (Origen: reggaeton)  Canción   
2         Farruko  Reggaeton (Origen: reggaeton)  Canción   
3          Plan B  Reggaeton (Origen: reggaeton)  Canción   
4     Cosculluela  Reggaeton (Origen: reggaeton)  Canción   

                          Nombre  Año de lanzamiento              ID_Spotify  \
0                        Tu Olor                2010  4lwJGmQFcbUypQ8mkm8YlG   
1      Zun Zun Rompiendo Caderas                2010  5aJshF6mtr7SsztEMhAvZv   
2               Su Hija Me Gusta                2010  5N1EdvOZKDKKIcwBcj3VmR   
3  Parisera (feat. De La Ghetto)                2010  4rVeA8n8ui7neX4XUOJObT   
4                Te Deseo el Mal                2010  43hrb3UDtUdvVNQ5gSCsMI   

                 ID_Album  Popularidad Playcount_La

In [24]:
# --- CELDA 6: FUSIÓN DE SPOTIFY (CARGADO DE CSV) + LAST.FM Y GUARDADO FINAL ---

import pandas as pd
import os

print("\n--- INICIANDO FUSIÓN DE DATOS (SPOTIFY + LAST.FM) ---")

# 1. CARGA DEL CSV INICIAL DE SPOTIFY
# CRÍTICO: Debe ser el nombre del archivo guardado en la Celda 3 (o la Celda 4, si la ejecutaste).
spotify_input_filename = 'reggaeton_data_2010_2024.csv' 
df_spotify_cargado = None # Inicializamos la variable

if os.path.exists(spotify_input_filename):
    df_spotify_cargado = pd.read_csv(spotify_input_filename)
    print(f"✅ DataFrame de Spotify cargado desde CSV: {len(df_spotify_cargado)} filas.")
else:
    print(f"❌ ERROR: No se encuentra el archivo CSV inicial de Spotify: {spotify_input_filename}")


if df_spotify_cargado is not None and 'df_lastfm' in locals():
    
    # 2. Realizamos la fusión (Merge) usando df_spotify_cargado como base
    df_proyecto_final = pd.merge(
        df_spotify_cargado, 
        df_lastfm,
        on='Artista', 
        how='left'
    )
    
    # 3. Limpieza de las columnas de Last.fm
    df_proyecto_final['Biografia_Resumen'] = df_proyecto_final['Biografia_Resumen'].fillna('N/A')
    
    # Rellenar valores nulos con 0 y asegurar que sean enteros
    for col in ['Playcount_LastFM', 'Listeners_LastFM']:
        if col in df_proyecto_final.columns:
            # Primero rellenamos NaN con 0.0 (float) y luego convertimos a int
            df_proyecto_final[col] = df_proyecto_final[col].fillna(0).astype(int)
    
    
    # 4. Guardar el archivo CSV FINAL
    output_filename = 'data_consolidada_final.csv'
    
    df_proyecto_final.to_csv(
        output_filename, 
        index=False, 
        encoding='utf-8'
    )
    
    print(f"\n🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉")
    print(f"Total de registros finales: {len(df_proyecto_final)}")
    print("--- Primeras 5 filas con datos clave ---")
    print(df_proyecto_final[['Nombre', 'Artista', 'Listeners_LastFM', 'Popularidad', 'Biografia_Resumen']].head())
    print(f"✅ Archivo final guardado: {output_filename}")
    
else:
    print("❌ ERROR DE FUSIÓN: Asegúrate de que la Celda 5 (Last.fm) se haya ejecutado exitosamente para crear 'df_lastfm'.")


--- INICIANDO FUSIÓN DE DATOS (SPOTIFY + LAST.FM) ---
✅ DataFrame de Spotify cargado desde CSV: 500 filas.

🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉
Total de registros finales: 500
--- Primeras 5 filas con datos clave ---
                          Nombre         Artista  Listeners_LastFM  \
0                        Tu Olor  Wisin & Yandel            823679   
1      Zun Zun Rompiendo Caderas  Wisin & Yandel            823679   
2               Su Hija Me Gusta         Farruko            772913   
3  Parisera (feat. De La Ghetto)          Plan B           1241301   
4                Te Deseo el Mal     Cosculluela            291143   

   Popularidad                                  Biografia_Resumen  
0           53  Wisin & Yandel is a Puerto Rican reggaeton duo...  
1           50  Wisin & Yandel is a Puerto Rican reggaeton duo...  
2           55  Farruko is a reggaeton and Latin pop singer fr...  
3           51  There are several artists and bands with this ...  
4           51  Born in

In [16]:
!pip install mysql-connector-python
# O usa el que suele ser más robusto con Pandas:
!pip install sqlalchemy mysqlclient